# Computes Graph and Text Embeddings, Elasticsearch Ready KGTK File and RDF Triples for Blazegraph

This notebook computes the following:

- `complEx` graph embeddings
- `transE` graph embeddings
- `BERT` text embeddings
- `elasticsearch` ready KGTK edge for [KGTK Search](https://kgtk.isi.edu/search/)
- `elasticsearch` ready KGTK edge file for Table Linker
- `RDF Triples` to be loaded into blazegraph

Inputs:

- `item_file`: the subset of the `claims_file` consistin of edges for property of data type `wikibase-item`
- `label_file`, `alias_file` and `description_file` containing labels, aliases and descriptions. It is assume that these files contain the labels, aliases and descriptions of all nodes appearing in the claims file. Users may provide these files for specific languages only.


### Batch Invocation
Example batch command. The second argument is a notebook where the output will be stored. You can load it to see progress.

```
papermill Embeddings-Elasticsearch-&-Triples.ipynb Embeddings-Elasticsearch-&-Triples.out.ipynb \
-p claims_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/all.tsv.gz \
-p label_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/part.label.en.tsv.gz \
-p item_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/part.wikibase-item.tsv.gz \
-p property_item_file = /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/part.property.wikibase-item.tsv.gz \
-p output_path <local folder> \
-p output_folder useful_files_v4 \
-p temp_folder temp.useful_files_v4 \
-p delete_database no 
-p languages es,ru,zh-cn
```

In [1]:
# Parameters
wikidata_root_folder = "/data/amandeep/wikidata-20210215-dwd-v2"
items_file = "claims.wikibase-item.tsv.gz"
all_sorted_file = "all.sorted.tsv.gz"
en_labels_file = "labels.en.tsv.gz"
temp_folder = "temp.elasticsearch.embeddings"

In [2]:
import os

In [3]:
os.environ['OUT'] = f"{wikidata_root_folder}"
os.environ['kgtk'] = "kgtk --debug"
os.environ['ITEMS'] = f"{wikidata_root_folder}/{items_file}"
os.environ['ALL'] = f"{wikidata_root_folder}/{all_sorted_file}"
os.environ['LABELS_EN'] = f"{wikidata_root_folder}/{en_labels_file}"

## Graph Embeddings

### complEx

In [6]:
complex_temp_folder = f"{wikidata_root_folder}/temp.graph-embeddings.complex"

In [7]:
!mkdir -p {complex_temp_folder}

In [ ]:
os.environ['TEMP_COMPLEX'] = complex_temp_folder

In [ ]:
!kgtk graph-embeddings --verbose -i "$ITEMS" \
-o $OUT/wikidatadwd.complEx.graph-embeddings.txt \
--retain_temporary_data True \
--operator ComplEx \
--workers 24 \
--log $TEMP_COMPLEX/ge.complex.log \
-T $TEMP_COMPLEX \
-ot w2v \
-e 600

### transE

In [ ]:
transe_temp_folder = f"{wikidata_root_folder}/temp.graph-embeddings.transe"

In [ ]:
!mkdir -p {transe_temp_folder}

In [ ]:
os.environ['TEMP_TRANSE'] = transe_temp_folder

In [ ]:
!$kgtk graph-embeddings --verbose -i "$ITEMS" \
-o $OUT/wikidatadwd.transE.graph-embeddings.txt \
--retain_temporary_data True \
--operator TransE \
--workers 24 \
--log $TEMP_TRANSE/ge.transE.log \
-T $TEMP_TRANSE \
-ot w2v \
-e 600

### BERT Embeddings

In [ ]:
!$kgtk text-embedding -i $ALL   \
--model roberta-large-nli-mean-tokens   \
--property-labels-file $LABELS_EN   \
--isa-properties P31 P279 P106 P39 P1382 P373 P452 \
--save-embedding-sentence > $OUT/wikidatadwd-text-embeddings-all.tsv

### Build KGTK edge file for KGTK Search

In [ ]:
 !$kgtk cat -i $OUT/all.sorted.tsv.gz \
    -i $OUT/derived.isastar.tsv.gz \
    -i $OUT/metadata.property.datatypes.tsv.gz \
    -i $OUT/metadata.pagerank.undirected.tsv.gz \
    -i $OUT/metadata.pagerank.directed.tsv.gz \
    -o $OUT/wikidata.dwd.all.kgtk.search.unsorted.tsv.gz

In [ ]:
!$kgtk sort -i $OUT/wikidata.dwd.all.kgtk.search.unsorted.tsv.gz \
--columns node1 \
 --extra '--parallel 24 --buffer-size 30% --temporary-directory ' + temp_folder_path \
-o $OUT/wikidata.dwd.all.kgtk.search.sorted.tsv.gz

### Build KGTK edge file for Triple generation


In [ ]:
!$kgtk cat \
-i $OUT/wikidata.dwd.all.kgtk.search.sorted.tsv.gz \
-i $OUT/derived.isa.tsv.gz \
-i $OUT/derived.P279star.tsv.gz \
-i $OUT/metadata.in_degree.tsv.gz \
-i $OUT/metadata.out_degree.tsv.gz \
-o $TEMP/wikidata.dwd.all.kgtk.triples.1.tsv.gz

In [ ]:
!$kgtk add-id -i $TEMP/wikidata.dwd.all.kgtk.triples.1.tsv.gz \
--id-style wikidata \
-o $TEMP/wikidata.dwd.all.kgtk.triples.2.tsv.gz

In [ ]:
!$kgtk sort -i $TEMP/wikidata.dwd.all.kgtk.triples.2.tsv.gz \
--columns node1 \
 --extra '--parallel 24 --buffer-size 30% --temporary-directory ' + temp_folder_path \
-o $OUT/wikidata.dwd.all.kgtk.triples.sorted.tsv.gz

Split the triples file to parallelize triple generation

In [ ]:
!mkdir -p $OUT/kgtk_triples_split

In [ ]:
!$kgtk split -i $OUT/wikidata.dwd.all.kgtk.triples.sorted.tsv.gz \
--output-path $OUT/kgtk_triples_split \
--gzipped-output --lines 10000000 \
--file-prefix kgtk_triples

In [ ]:
!curl https://raw.githubusercontent.com/usc-isi-i2/kgtk/dev/kgtk-properties/kgtk.properties.tsv -o $TEMP/kgtk-properties.tsv

In [ ]:
!$kgtk filter -p ";data_type;" -i $TEMP/kgtk-properties.tsv -o $TEMP/kgtk-properties.datatype.tsv.gz

In [1]:
!$kgtk cat -i $TEMP/kgtk-properties.datatype.tsv.gz $OUT/metadata.property.datatypes.tsv.gz -o $OUT/metadata.property.datatypes.augmented.tsv.gz

cat: illegal option -- i
usage: cat [-benstuv] [file ...]


In [ ]:
!ls $OUT/kgtk_triples_split/*tsv.gz | parallel -j 18 'kgtk generate-wikidata-triples -lp label -ap alias -dp description -pf $OUT/metadata.property.datatypes.augmented.tsv.gz --output-n-lines 100000 --generate-truthy --warning --use-id --log-path $TEMP/generate_triples_log.txt --error-action log -i {} > {.}.ttl'



### Build KGTK edge file for Table Linker Index

The augmentation files can be downloaded from Google Drive,
https://drive.google.com/drive/u/1/folders/1qbbgjo7pddMdDvQzOSeSaL6lYwj_f5gi

In [ ]:
def convert_w2v_kgtk(input_path, output_path, label):
    f = open(input_path)
    o = gzip.open(output_path, 'wt')
    o.write("node1\tlabel\tnode2\n")
    i = 0
    for line in f:
        if i % 1000000 == 0:
            print(i)
        i += 1
        vals = line.strip().split(" ")
        if len(vals) == 2: # first line
            continue
        qnode = vals[0]
        vector = ",".join(vals[1:])
        o.write(f"{qnode}\t{label}\t{vector}\n")
    print("Done!")

In [ ]:
convert_w2v_kgtk("$OUT/wikidatadwd.complEx.graph-embeddings.txt", "$OUT/wikidatadwd.complEx.graph-embeddings.tsv.gz", "graph_embeddings_complEx")

In [ ]:
convert_w2v_kgtk('$OUT/wikidatadwd.transE.graph-embeddings.txt', '$OUT/wikidatadwd.transE.graph-embeddings.tsv.gz', 'graph_embeddings_transE')

In [ ]:
!$kgtk cat -i $OUT/wikidata.dwd.all.kgtk.search.sorted.tsv.gz \
/data/amandeep/wikidata_augmented_files_for_table_linker_index/augmentation.wikipedia.anchors.tsv.gz \
/data/amandeep/wikidata_augmented_files_for_table_linker_index/augmentation.wikipedia.tables.anchors.tsv.gz \
/data/amandeep/wikidata_augmented_files_for_table_linker_index/augmentation.wikipedia.redirect.tsv.gz \
$OUT/item.property.count.compact.tsv.gz \
$OUT/dwd_isa_class_count.compact.tsv.gz \
$OUT/wikidatadwd.complEx.graph-embeddings.tsv.gz \
$OUT/wikidatadwd.transE.graph-embeddings.tsv.gz \
$OUT/text-embeddings-concatenated.tsv.gz \
$OUT/derived.dwd_isa.tsv.gz \
$OUT/table_linker.qnode.property.values.tsv.gz \
$OUT/derived.isastar.tsv.gz \
-o $TEMP/wikidata.dwd.table-linker.index.tsv.gz

In [ ]:
!$kgtk sort -i $TEMP/wikidata.dwd.table-linker.index.tsv.gz \
--columns node1 \
 --extra '--parallel 24 --buffer-size 30% --temporary-directory ' + temp_folder_path \
-o $OUT/wikidata.dwd.table-linker.index.sorted.tsv.gz

In [ ]:
!$kgtk build-kgtk-search-input --input-file $OUT/wikidata.dwd.table-linker.index.sorted.tsv.gz  \
--output-file $OUT/wikidata.dwd.table-linker.index.sorted.jl \
--label-properties label \
--alias-properties alias \
--description-properties description \
--pagerank-properties directed_pagerank \
--mapping-file $OUT/wikidata_dwd.v2.table-linker.json \
--property-datatype-file $OUT/metadata.property.datatypes.augmented.tsv.gz